# Data Preprocessing

## Import libraries

In [27]:
import pandas as pd

In [28]:
df = pd.read_csv('data/output/output_dataset.csv')

df.head()

,code,url,titre,contenu,langue,nb_caracteres,nb_mots,date_publication,lieu,maladie,animal,source_publication,resume_50_mots,resume_100_mots,resume_150_mots
0,code151,https://lc.cx/nKVbsM,Bluetongue virus was detected in Europe,Tridge LogoGlobal trading platform Tridge Get ...,en,3891,617,16/10/2023,Allemagne,Bluetongue,"Ovins, Bovins",lien raccourci,Le virus de la fièvre catarrhale ovine (Blueto...,Le virus de la fièvre catarrhale ovine (Blueto...,Le virus de la fièvre catarrhale ovine (Blueto...
1,code152,https://lc.cx/sXWRhi,Bluetongue officially confirmed in Germany,Tridge LogoGlobal trading platform Tridge Get ...,en,3942,610,17 octobre 2023,Allemagne,Bluetongue,"Bovins, Ovins",lien raccourci,Un animal bovin a été diagnostiqué positif pou...,La maladie a été détectée après les Pays-Bas e...,La détection du virus de la fièvre catarrhale ...
2,code153,https://www.agriland.ie/farming-news/daera-urg...,DAERA urges farmers to be vigilant for blueton...,Latest News Beef Dairy Sheep Tillage Machinery...,en,4947,770,18 octobre 2023,Irlande du Nord,Bluetongue,"Bovins, Ovins",site officiel,Le DAERA de l'Irlande du Nord a mis en garde l...,Le DAERA de l'Irlande du Nord a mis en garde l...,Le DAERA de l'Irlande du Nord a mis en garde l...
3,code154,https://lc.cx/fnYicq,Erreur,Erreur: Message: unknown error: net::ERR_NAME_...,en,566,42,19 octobre 2023,Allemagne,Bluetongue,"Ovins, Bovins",lien raccourci,Le virus de la fièvre catarrhale ovine (Blueto...,Le virus de la fièvre catarrhale ovine (Blueto...,Le virus de la fièvre catarrhale ovine (Blueto...
4,code155,https://lc.cx/V_Nr3n,اقتصاد,بعد موجة بق الفراش، ترزح العديد من الدول الأور...,ar,1169,191,19 octobre 2023,France,Bluetongue,"Ovins, Bovins",lien raccourci,La France détecte des cas de Bluetongue chez l...,Le virus de la fièvre catarrhale ovine (Blueto...,La France a déclaré un état d'urgence après le...


In [29]:
print(df['maladie'])

0                                            Bluetongue
1                                            Bluetongue
2                                            Bluetongue
3                                            Bluetongue
4                                            Bluetongue
5                                            Bluetongue
6                                            Bluetongue
7                  Fièvre catarrhale ovine (Bluetongue)
8                       Dermatose nodulaire contagieuse
9                      Maladie hémorragique épizootique
10               Maladie hémorragique épizootique (MHE)
11                     Maladie hémorragique épizootique
12                      Dermatose nodulaire contagieuse
13                                           Bluetongue
14                Dermatose nodulaire contagieuse (DNC)
15                                           Bluetongue
16                     Maladie hémorragique épizootique
17                                             i

In [30]:
import re
from difflib import SequenceMatcher
disease_mapping = {
    # Bluetongue / Fièvre catarrhale ovine
    'bluetongue': 'Fièvre catarrhale ovine (Bluetongue)',
    'fièvre catarrhale ovine': 'Fièvre catarrhale ovine (Bluetongue)',
    'fco': 'Fièvre catarrhale ovine (Bluetongue)',
    'langue bleue': 'Fièvre catarrhale ovine (Bluetongue)',
    'bluetongue (inf. par le virus de la)': 'Fièvre catarrhale ovine (Bluetongue)',
    
    # Maladie hémorragique épizootique
    'maladie hémorragique épizootique': 'Maladie hémorragique épizootique (MHE)',
    'mhe': 'Maladie hémorragique épizootique (MHE)',
    'maladie hémorragique épizootique (inf. par le virus de la)': 'Maladie hémorragique épizootique (MHE)',
    
    # Dermatose nodulaire contagieuse
    'dermatose nodulaire contagieuse': 'Dermatose nodulaire contagieuse',
    'dnc': 'Dermatose nodulaire contagieuse',
    'dermatose nodulaire contagieuse (inf. par le virus de la)': 'Dermatose nodulaire contagieuse',
    
    # West Nile / Fièvre du Nil occidental
    'fièvre de west nile': 'Fièvre du Nil occidental (West Nile)',
    'west nile virus': 'Fièvre du Nil occidental (West Nile)',
    'wnv': 'Fièvre du Nil occidental (West Nile)',
    'virus du nil occidental': 'Fièvre du Nil occidental (West Nile)',
    
    # Influenza équine
    'grippe équine': 'Influenza équine',
    'grippe équine (inf. par le virus de la)': 'Influenza équine',
    'equine influenza': 'Influenza équine',
    
    # SARS-CoV-2
    'sars-cov-2': 'SARS-CoV-2 chez les animaux',
    'sars-cov-2 chez les animaux': 'SARS-CoV-2 chez les animaux',
    'sars-cov-2 chez les animaux (inf. par le)': 'SARS-CoV-2 chez les animaux',
    
    # Autres maladies
    'tuberculose': 'Tuberculose bovine',
    'clavelée et variole caprine': 'Clavelée et variole caprine',
    'nécrose hématopoïétique infectieuse': 'Nécrose hématopoïétique infectieuse',
    'aethina tumida': 'Petit coléoptère des ruches (Aethina tumida)',
    'ppr': 'Peste des Petits Ruminants (PPR)',
    'ppcb': 'Péripneumonie Contagieuse Bovine (PPCB)',
}

def normalize_disease_name(disease_name):
    """
    Normalise le nom d'une maladie
    """
    if pd.isna(disease_name):
        return 'Maladie non spécifiée'
    
    # Convertir en minuscules et nettoyer
    disease_clean = str(disease_name).lower().strip()
    
    # Chercher dans le dictionnaire
    if disease_clean in disease_mapping:
        return disease_mapping[disease_clean]
    
    # Chercher une correspondance partielle
    for key, value in disease_mapping.items():
        if key in disease_clean or disease_clean in key:
            return value
    
    # Si aucune correspondance, retourner le nom original capitalisé
    return disease_name.strip().title()

# Appliquer la normalisation
df['maladie_normalisee'] = df['maladie'].apply(normalize_disease_name)

# Afficher les résultats
print("=== Statistiques avant normalisation ===")
print(f"Nombre de maladies uniques : {df['maladie'].nunique()}")
print("\nDistribution des maladies :")
print(df['maladie'].value_counts())

print("\n=== Statistiques après normalisation ===")
print(f"Nombre de maladies uniques : {df['maladie_normalisee'].nunique()}")
print("\nDistribution des maladies normalisées :")
print(df['maladie_normalisee'].value_counts())

# Afficher un tableau de correspondance
print("\n=== Tableau de correspondance ===")
correspondance = df[['maladie', 'maladie_normalisee']].drop_duplicates().sort_values('maladie_normalisee')
print(correspondance)

# Sauvegarder le résultat
df.to_csv('data/output/donnees_normalisees.csv', index=False)
print("\n✅ Fichier sauvegardé : data/output/donnees_normalisees.csv")

# Fonction optionnelle pour ajouter de nouvelles correspondances
def add_disease_mapping(original, normalized):
    """
    Ajoute une nouvelle correspondance au dictionnaire
    """
    disease_mapping[original.lower().strip()] = normalized
    print(f"✅ Ajouté : '{original}' -> '{normalized}'")


=== Statistiques avant normalisation ===
Nombre de maladies uniques : 21

Distribution des maladies :
maladie
Bluetongue                                                                     15
Dermatose nodulaire contagieuse                                                 6
Maladie hémorragique épizootique                                                6
Fièvre de West Nile                                                             3
Fièvre catarrhale ovine (Bluetongue)                                            3
SARS-CoV-2 chez les animaux (Inf. par le)                                       2
Clavelée et variole caprine                                                     1
Infection à virus West Nile                                                     1
Fièvre hémorragique de Crimée-Congo                                             1
Fièvre aphteuse (Foot and mouth disease)                                        1
Nécrose hématopoïétique infectieuse (Inf. par le virus de la)         

In [31]:
df.head()

,code,url,titre,contenu,langue,nb_caracteres,nb_mots,date_publication,lieu,maladie,animal,source_publication,resume_50_mots,resume_100_mots,resume_150_mots,maladie_normalisee
0,code151,https://lc.cx/nKVbsM,Bluetongue virus was detected in Europe,Tridge LogoGlobal trading platform Tridge Get ...,en,3891,617,16/10/2023,Allemagne,Bluetongue,"Ovins, Bovins",lien raccourci,Le virus de la fièvre catarrhale ovine (Blueto...,Le virus de la fièvre catarrhale ovine (Blueto...,Le virus de la fièvre catarrhale ovine (Blueto...,Fièvre catarrhale ovine (Bluetongue)
1,code152,https://lc.cx/sXWRhi,Bluetongue officially confirmed in Germany,Tridge LogoGlobal trading platform Tridge Get ...,en,3942,610,17 octobre 2023,Allemagne,Bluetongue,"Bovins, Ovins",lien raccourci,Un animal bovin a été diagnostiqué positif pou...,La maladie a été détectée après les Pays-Bas e...,La détection du virus de la fièvre catarrhale ...,Fièvre catarrhale ovine (Bluetongue)
2,code153,https://www.agriland.ie/farming-news/daera-urg...,DAERA urges farmers to be vigilant for blueton...,Latest News Beef Dairy Sheep Tillage Machinery...,en,4947,770,18 octobre 2023,Irlande du Nord,Bluetongue,"Bovins, Ovins",site officiel,Le DAERA de l'Irlande du Nord a mis en garde l...,Le DAERA de l'Irlande du Nord a mis en garde l...,Le DAERA de l'Irlande du Nord a mis en garde l...,Fièvre catarrhale ovine (Bluetongue)
3,code154,https://lc.cx/fnYicq,Erreur,Erreur: Message: unknown error: net::ERR_NAME_...,en,566,42,19 octobre 2023,Allemagne,Bluetongue,"Ovins, Bovins",lien raccourci,Le virus de la fièvre catarrhale ovine (Blueto...,Le virus de la fièvre catarrhale ovine (Blueto...,Le virus de la fièvre catarrhale ovine (Blueto...,Fièvre catarrhale ovine (Bluetongue)
4,code155,https://lc.cx/V_Nr3n,اقتصاد,بعد موجة بق الفراش، ترزح العديد من الدول الأور...,ar,1169,191,19 octobre 2023,France,Bluetongue,"Ovins, Bovins",lien raccourci,La France détecte des cas de Bluetongue chez l...,Le virus de la fièvre catarrhale ovine (Blueto...,La France a déclaré un état d'urgence après le...,Fièvre catarrhale ovine (Bluetongue)


In [32]:
df_clean = df[
    ~df['titre'].str.contains('page non trouvée|404|erreur|not found', case=False, na=False) &
    ~df['maladie'].str.contains('inconnu', case=False, na=False) &
    df['maladie'].notna()
]

print(f"Lignes après nettoyage : {len(df_clean)}")
print(f"Lignes supprimées : {len(df) - len(df_clean)}")

# Sauvegarder
df_clean.to_csv('data/output/dataset_nettoye.csv', index=False)
print("✅ Dataset nettoyé sauvegardé !")

Lignes après nettoyage : 45
Lignes supprimées : 5
✅ Dataset nettoyé sauvegardé !


In [33]:
df.columns

Index(['code', 'url', 'titre', 'contenu', 'langue', 'nb_caracteres', 'nb_mots',
       'date_publication', 'lieu', 'maladie', 'animal', 'source_publication',
       'resume_50_mots', 'resume_100_mots', 'resume_150_mots',
       'maladie_normalisee'],
      dtype='object')

In [37]:
from deep_translator import GoogleTranslator
from langdetect import detect, LangDetectException
import time
from tqdm import tqdm
import re

# Charger le dataset
df = pd.read_csv('data/output/dataset_nettoye.csv')

translator = GoogleTranslator(source='auto', target='fr')

# Dictionnaire de traduction pour les noms d'animaux (translittération arabe -> français)
ANIMAUX_DICT = {
    # Translittérations arabes vers français
    'aghnam': 'ovins',
    'agnam': 'ovins',
    'abqar': 'bovins',
    'abkar': 'bovins',
    'maaez': 'caprins',
    'maez': 'caprins',
    'maàz': 'caprins',
    'dajaj': 'volailles',
    'djaj': 'volailles',
    'khayl': 'équins',
    'jamal': 'camélidés',
    'himaar': 'ânes',
    
    # Mots arabes en caractères arabes
    'أغنام': 'ovins',
    'أبقار': 'bovins',
    'ماعز': 'caprins',
    'دجاج': 'volailles',
    'خيل': 'équins',
    'جمال': 'camélidés',
    'حمير': 'ânes',
    
    # Pluriels et variantes
    'sheep': 'ovins',
    'cattle': 'bovins',
    'goat': 'caprins',
    'goats': 'caprins',
    'poultry': 'volailles',
    'chicken': 'volailles',
}

def contient_arabe(texte):
    """Détecte si le texte contient des caractères arabes"""
    if pd.isna(texte) or str(texte).strip() == '':
        return False
    pattern = re.compile(r'[\u0600-\u06FF\u0750-\u077F\u08A0-\u08FF]+')
    return bool(pattern.search(str(texte)))

def traduire_noms_animaux(texte):
    """Traduit les noms d'animaux via le dictionnaire personnalisé"""
    if pd.isna(texte) or str(texte).strip() == '':
        return texte
    
    texte_traduit = str(texte)
    
    # Séparer par virgules s'il y a plusieurs animaux
    animaux = [a.strip() for a in texte_traduit.split(',')]
    animaux_traduits = []
    
    for animal in animaux:
        animal_lower = animal.lower().strip()
        
        # Chercher dans le dictionnaire
        traduit = False
        for key, value in ANIMAUX_DICT.items():
            if key in animal_lower or animal_lower in key:
                animaux_traduits.append(value.capitalize())
                traduit = True
                break
        
        # Si pas trouvé dans le dictionnaire, garder tel quel
        if not traduit:
            animaux_traduits.append(animal)
    
    return ', '.join(animaux_traduits)

def est_francais(texte):
    """Détecte si le texte est déjà en français"""
    if pd.isna(texte) or str(texte).strip() == '':
        return True
    
    # Si contient de l'arabe, ce n'est pas français
    if contient_arabe(texte):
        return False
    
    try:
        echantillon = str(texte)[:200].strip()
        if len(echantillon) < 3:
            return True
        langue = detect(echantillon)
        return langue == 'fr'
    except LangDetectException:
        return True
    except Exception:
        return True

def traduire_si_necessaire(texte, nom_colonne=''):
    """Traduit uniquement si ce n'est pas en français"""
    if pd.isna(texte):
        return texte
    
    texte_str = str(texte).strip()
    
    if texte_str == '':
        return texte_str
    
    # Traitement spécial pour la colonne 'animal'
    if nom_colonne.lower() == 'animal':
        return traduire_noms_animaux(texte_str)
    
    # Vérifier si déjà en français
    if est_francais(texte_str):
        return texte_str
    
    try:
        texte_limite = texte_str[:4999]
        traduction = translator.translate(texte_limite)
        time.sleep(0.3)
        return traduction
    
    except Exception as e:
        print(f"\n⚠️ Erreur lors de la traduction de '{nom_colonne}': {e}")
        print(f"   Texte original: {texte_str[:100]}...")
        return texte_str

# Traduire les colonnes
colonnes_a_traduire = ['titre', 'contenu', 'lieu', 'maladie', 'animal', 
                       'resume_50_mots', 'resume_100_mots', 'resume_150_mots']

print("🚀 Début de la traduction du dataset\n")

for colonne in colonnes_a_traduire:
    if colonne not in df.columns:
        print(f"⚠️ Colonne '{colonne}' non trouvée, passage à la suivante...")
        continue
    
    print(f"\n{'='*60}")
    print(f"🔄 Traitement de la colonne '{colonne}'")
    print(f"{'='*60}")
    
    # Afficher quelques exemples avant traduction (pour la colonne animal)
    if colonne.lower() == 'animal':
        print(f"\n📋 Exemples avant traduction:")
        exemples = df[colonne].dropna().unique()[:5]
        for ex in exemples:
            print(f"   • {ex}")
    
    # Compter combien nécessitent une traduction
    if colonne.lower() == 'animal':
        # Pour les animaux, toujours traiter via le dictionnaire
        nb_a_traduire = len(df[df[colonne].notna()])
    else:
        non_francais = df[colonne].apply(lambda x: not est_francais(x))
        nb_a_traduire = non_francais.sum()
    
    total = len(df)
    
    print(f"\n📊 Statistiques:")
    print(f"   - Total d'entrées: {total}")
    print(f"   - À traiter: {nb_a_traduire}")
    
    # Détecter les textes avec arabe
    avec_arabe = df[colonne].apply(contient_arabe).sum()
    if avec_arabe > 0:
        print(f"   - Contenant de l'arabe: {avec_arabe}")
    
    if nb_a_traduire > 0:
        print(f"\n⏳ Traduction en cours...")
        df[colonne] = [
            traduire_si_necessaire(texte, colonne)
            for texte in tqdm(df[colonne], desc=f"  {colonne}", 
                            unit="texte", ncols=80)
        ]
        
        # Afficher quelques exemples après traduction (pour la colonne animal)
        if colonne.lower() == 'animal':
            print(f"\n✨ Exemples après traduction:")
            exemples = df[colonne].dropna().unique()[:5]
            for ex in exemples:
                print(f"   • {ex}")
        
        print(f"✅ '{colonne}' terminée !")
    else:
        print(f"✓ Aucune traduction nécessaire pour '{colonne}'")

print(f"\n{'='*60}")
print("💾 Sauvegarde du dataset traduit...")

# Sauvegarder
output_path = 'data/output/dataset.csv'
df.to_csv(output_path, index=False, encoding='utf-8')

print(f"✅ Traduction terminée et sauvegardée dans '{output_path}'!")
print(f"📊 Dataset final: {len(df)} lignes × {len(df.columns)} colonnes")

# Afficher un résumé des traductions d'animaux
print(f"\n{'='*60}")
print("🐑 Résumé des noms d'animaux traduits:")
print(f"{'='*60}")
if 'animal' in df.columns:
    animaux_uniques = df['animal'].dropna().unique()
    for animal in sorted(animaux_uniques):
        print(f"   • {animal}")

🚀 Début de la traduction du dataset


🔄 Traitement de la colonne 'titre'

📊 Statistiques:
   - Total d'entrées: 45
   - À traiter: 32
   - Contenant de l'arabe: 10

⏳ Traduction en cours...


  titre: 100%|███████████████████████████████| 45/45 [00:23<00:00,  1.88texte/s]


✅ 'titre' terminée !

🔄 Traitement de la colonne 'contenu'

📊 Statistiques:
   - Total d'entrées: 45
   - À traiter: 17
   - Contenant de l'arabe: 10

⏳ Traduction en cours...


  contenu:  56%|████████████████             | 25/45 [00:08<00:05,  3.60texte/s]


⚠️ Erreur lors de la traduction de 'contenu': Request exception can happen due to an api connection error. Please check your connection and try again
   Texte original: الرئيسية أخبار المحافظات تحصين 79 ألفا و222 رأس أغنام وماعز للوقاية من طاعون المجترات الصغيرة بالشرق...


...

✅ 'resume_50_mots' terminée !

🔄 Traitement de la colonne 'resume_100_mots'

📊 Statistiques:
   - Total d'entrées: 45
   - À traiter: 1
   - Contenant de l'arabe: 1

⏳ Traduction en cours...


  resume_100_mots: 100%|█████████████████████| 45/45 [00:00<00:00, 66.12texte/s]


✅ 'resume_100_mots' terminée !

🔄 Traitement de la colonne 'resume_150_mots'

📊 Statistiques:
   - Total d'entrées: 45
   - À traiter: 1
   - Contenant de l'arabe: 1

⏳ Traduction en cours...


  resume_150_mots: 100%|█████████████████████| 45/45 [00:00<00:00, 60.62texte/s]

✅ 'resume_150_mots' terminée !

💾 Sauvegarde du dataset traduit...
✅ Traduction terminée et sauvegardée dans 'data/output/dataset.csv'!
📊 Dataset final: 45 lignes × 16 colonnes

🐑 Résumé des noms d'animaux traduits:
   • Abeilles (DOMESTIQUE)
   • Abovins, Douans
   • Abpars (Bos taurus)
   • Agnans, Moutons, Doves
   • Bovins
   • Bovins, Buffles
   • Bovins, Ovins
   • Bovins, Ovins, Caprins, Cervidés d'élevage
   • Bovins, Têtes de bétail
   • Bovins, Élaphe (SAUVAGE), Daim européen (SAUVAGE), Ovins (DOMESTIQUE)
   • Bovins, Équidés
   • Caprins
   • Equidés
   • Homme
   • Mashia, طائر
   • Mustelidae (DOMESTIQUE)
   • Ovins, Bovins
   • Ovins, Bovins, Équidés, Volailles
   • Ovins, Caprins
   • Ovins, Masse
   • Petits ruminants, Bovins
   • Porcins, Bovins
   • Truite arc-en-ciel (DOMESTIQUE)
   • Équidés
   • Équidés, Chevaux, Oiseaux
